
---

---

# examples/fmri_simulation_tutorial.ipynb

---

## Title + intro → MARKDOWN:


# PySimPace: fMRI Simulation Tutorial

This notebook demonstrates how to use PySimPace to simulate **motion artifacts** and **physiological artifacts** in functional MRI (4D volumes).

We will show:

✅ Simulating fMRI motion:
- Inter-volume motion
- Intra-volume motion (slice-wise)
- Physiological noise

✅ Visualizing motion effects  
✅ Using MotionTrajectory API  

---


In [1]:
import os
from pysimpace.io import load_nifti, save_nifti
from pysimpace.simulation.functional import simulate_fmri_motion
from pysimpace.simulation.models import MotionTrajectory, generate_smooth_motion_trajectory
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [ ]:
# Path to a clean fMRI scan
clean_path = os.path.join("examples", "clean", "sub-01_task-deterministicclassification_run-01_bold.nii.gz")

fmri_data, affine, header = load_nifti(clean_path)
print("fMRI shape:", fmri_data.shape)


In [ ]:
#motion_trajectory
n_vols = fmri_data.shape[-1]
n_slices = fmri_data.shape[2]

trajectory = MotionTrajectory(n_volumes=n_vols, n_slices=n_slices)

volume_transforms = generate_smooth_motion_trajectory(
    n_volumes=n_vols,
    target_fd_mm=0.5,
    vol_shape=fmri_data.shape[:3],
    smoothing_sigma=3.0,
    seed=42
)

for t in range(n_vols):
    trajectory.set_volume_transform(t, volume_transforms[t])


In [ ]:
#simulation
corrupted_fmri = simulate_fmri_motion(
    fmri_data,
    trajectory,
    intra=True,
    physio=True,
    parallel=True,
    seed=42
)